In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import openai
from dotenv import load_dotenv, find_dotenv
import os
import sys

sys.path.append("../..")


_ = load_dotenv(find_dotenv())  # read local .env file

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
import langchain

langchain.debug = False

In [4]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings, OpenAIEmbeddings

embeddings_hf = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    api_key=HUGGINGFACEHUB_API_TOKEN,
)

embeddings_openai = OpenAIEmbeddings()

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import nest_asyncio
page = None
async def main(url):
    global page
    browser = await launch()
    page = await browser.newPage()
    await page.goto(url)
    
    # Get the page content after JavaScript execution
    page_content = await page.content()

    # Close the browser
    await browser.close()

    # Now you can use BeautifulSoup to parse the page content
    soup = BeautifulSoup(page_content, 'html.parser')

    # Perform scraping using BeautifulSoup on the dynamically rendered content
    # ...return soup
    # print(soup.prettify())
    return soup

nest_asyncio.apply()
url = "https://www.mdcsoftware.com.vn/"
soup = asyncio.get_event_loop().run_until_complete(main(url))



In [7]:
from bs4 import BeautifulSoup
recruiment = BeautifulSoup()
block = "MDC SoftwareYou need to enable JavaScript to run this app."

In [8]:
while not recruiment.text or recruiment.text == block:
    recruiment = asyncio.get_event_loop().run_until_complete(main(url+"recruitment"))

In [11]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("h5", "Header 5"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

html_header_splits = html_splitter.split_text(soup.text[:12] + soup.text[58:] + "\n" + recruiment.text[113:-225])

chunk_size = 200
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

docs = text_splitter.split_documents(html_header_splits)

In [16]:
from agentic_chunking import get_propositions

propositions = get_propositions(soup.text[:12] + soup.text[58:] + "\n" + recruiment.text[113:-225])

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
propositions

['About Us: MDC Software is a technology company specializing in product design and development for high growth companies.',
 'Services and Products: MDC Software offers services such as web application development, artificial intelligence, internet of things, UX design, streaming solutions, product marketing, payment integration, blockchain, technical advice, and more. They also have developed products such as iVPN, iMatch, Advanced IPTV Player, and Can Knockdown.',
 'Address: MDC VIETNAM MEDIA AND TECHNOLOGY COMPANY LIMITED, 15th Floor, Viet A Building, No. 9 Duy Tan, Cau Giay, Hanoi',
 'Contact: 0868.733.900, hello@mdcsoftware.com.vn',
 'Recruitment: MDC Media is currently recruiting front-end react native developers and freshers for new projects. The job description includes JavaScript programming, participation in project development, and application development on platforms such as React Native and ReactJS.',
 "Mission: MDC Software's mission is to create new value for customers 

## FAISS db

In [34]:
from langchain_community.vectorstores import FAISS

db = await FAISS.afrom_texts(propositions,embeddings_openai)

## Elastic Search

In [10]:
ELS_API_KEY = os.environ["ELS_API_KEY"]
ELS_CLOUD_ID = os.environ["ELS_CLOUD_ID"]

In [14]:
import elasticsearch
from langchain_elasticsearch import ElasticsearchStore

es_client = elasticsearch.Elasticsearch(
    cloud_id=ELS_CLOUD_ID,
    api_key=ELS_API_KEY,
    max_retries=5,
)
elastic_vector_search = ElasticsearchStore(
    index_name="mdc",
    es_connection=es_client,
    embedding=embeddings_openai
)

In [18]:
elastic_vector_search.add_texts(propositions)


['54012d33-df7b-495a-832e-e644bfb55a7b',
 '79a94acd-1b18-4a97-addc-cae195e2a0fd',
 '4179e2d6-3664-4328-b4d8-2294f1bdb617',
 '1ea02892-337c-49c4-8432-a74946dc6489',
 '43d08a69-68b3-492f-bebf-d03d14bc728c',
 '83ed8388-1514-439a-be35-ad95645a5fd3',
 'daaf4f85-8e4d-4a1b-8d3a-f6a9bbecc83b']

In [23]:
from langchain.chat_models import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough,
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

retriever = elastic_vector_search.as_retriever(search_type="mmr", search_kwargs={"k": 2})

template = """You're helpful assistant, please answer the question based only on the following context:
{context}
Your response based on user's question language.
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0,
    callbacks=[StreamingStdOutCallbackHandler()],
    streaming=True,
)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
chain.invoke("Công ty này ở Sài Gòn à?")

Based on the information provided, the company is located in Hanoi, not in Saigon.

'Based on the information provided, the company is located in Hanoi, not in Saigon.'

In [27]:
from huggingface_hub import login

login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.